# Chat-bot in the Telegram

In [1]:
import random
import nltk

In [2]:
with open('final_config.txt', 'r', encoding='utf-8') as f:
    big_config = f.read()
    f.close()
#big_config

FileNotFoundError: [Errno 2] No such file or directory: 'final_config.txt'

In [3]:
# убираем мусор из текстового файла
if '\ufeff' in big_config:
        big_config = big_config.split('\ufeff')[1]
# преобразуем text в json
big_config = eval(big_config)
#big_config

NameError: name 'big_config' is not defined

In [4]:
# набор данных на вход и на выход

BOT_CONFIG = big_config

In [5]:
# вытащим интенты и их значения(example)
dataset = []

for intent, intent_data in BOT_CONFIG['intents'].items():
    for example in intent_data['examples']:
        dataset.append([example, intent])

In [6]:
# задаём объекты
X_text = [x for x, y in dataset]
# задаём соответствующие им классы
y = [y for x, y in dataset]

In [7]:
# подключаем библиотеку для векторизации
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer='char', ngram_range=(3, 3)) 
X = vectorizer.fit_transform(X_text)

In [8]:
# подключаем библиотеку для обучения модели
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()

# Так как проверка прошла успешно, поэтому обучаем на всём наборе данных
clf.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [9]:
def get_intent(text):
    # Список вероятностей попадания вопроса в каждый класс
    probas = clf.predict_proba(vectorizer.transform([text]))[0]
    # ищем максимальную вероятность попадания
    proba = max(probas)
    # и проверяем её на пригодность
    if proba > 0.3:
        # определяем индекс класса, к которому принадлежит ответ
        index = list(probas).index(proba)
        # возвращаем данный класс
        return clf.classes_[index]

In [10]:
with open('dialogues.txt', 'r', encoding='utf-8') as f:
    dialogues_data = f.read()
    f.close()

In [11]:
# составляем общий список диалогов, убираем абзацы и переносы на следующую строку
dialogues = [dialogue.split('\n')[:2] for dialogue in dialogues_data.split('\n\n')]

In [12]:
# оставляем только те диалоги, которые состоят из двух реплик
dialogues = [dialogue for dialogue in dialogues if len(dialogue) ==2]

In [13]:
dialogues_filtered = []
# убираем мусор, кроме тех символов, которые нужны
alphabet = '1234567890- абвгдеёжзийклмнопрстуфхцчшщъыьэюяqwertyuiopasdfghjklzxcvbnm'

for dialogue in dialogues:
    # Пропускаем первые два символа в реплике(- ) и убираем заглавную букву
    question = dialogue[0][2:].lower()
    # Оставляем в цикле реплики, которые состоят из нужных символов, и соединяем их в предложение с помощью join в tuple
    question = ''.join(char for char in question if char in alphabet)
    # убираем пробелы до и после реплики
    question = question.strip()
    # убираем пробелы до и после реплики
    answer = dialogue[1][2:].strip()
    if question and answer:
        dialogues_filtered.append((question, answer))

In [14]:
# убираем повторы
dialogues_filtered = list(set(dialogues_filtered))

In [15]:
# Для оптимизации поиска правильного ответа, было решено разбить вопросы на ключевые слова
search_structure = {}  # {word: [(q, a), (q, a), ...], ...}

for question, answer in dialogues_filtered:
    # разбиваем вопрос на ключевые слова
    words = question.split(' ')
    for word in words:
        if word not in search_structure:
            search_structure[word] = []
        search_structure[word].append((question, answer))

# Чтобы не было путаницы между популярными ключевыми словами, было решено их исключить
to_del = []
for word in search_structure:
    if len(search_structure[word]) > 10000:
        to_del.append(word)

# Удаление ключевого слова, которое слишком популярно
for word in to_del:
    search_structure.pop(word)

In [16]:
def get_generative_response(text):
    # Убираем заглавие
    text = text.lower()
     # Оставляем в цикле реплики, которые состоят из нужных символов, и соединяем их в предложение с помощью join в tuple
    text = ''.join(char for char in text if char in alphabet)
     # убираем пробелы до и после реплики
    text = text.strip()
     # разбиваем вопрос на ключевые слова
    words = text.split(' ')

    # Для ускорения поиска ответа было решено искать ответы, которые подходят по ключевым словам в вопросе
    qas = []
    for word in words:
        if word in search_structure:
            qas += search_structure[word]
            
   # Для ускорения выдачи ответа было решено ограничить порог входа с помощью расстояния Левенштейна
    for question, answer in qas:
        if abs(len(text) - len(question)) < len(question) * 0.20:
            edit_distance = nltk.edit_distance(text, question)
            if edit_distance / len(question) < 0.20:
                return answer

In [17]:
# выбор ответа по закону рандома на фразы, которые есть в наборе данных
def get_response_by_intent(intent):
    phrases = BOT_CONFIG['intents'][intent]['responses']
    return random.choice(phrases)

# выбор ответа по закону рандома на фразы, которых нет в наборе данных 
def get_failure_phrases():
    phrases = BOT_CONFIG['failure_phrases']
    return random.choice(phrases)

In [18]:
# Статистика использования моделей(функций)
stats = {'intent': 0, 'generative': 0, 'stub': 0}

In [19]:
def bot(text):   
    """Генерация ответной реплики"""
    
    # NLU - распознавание
    intent = get_intent(text)
    # rules
    if intent:
        stats['intent'] += 1
        return get_response_by_intent(intent)

    # use generative model
    response = get_generative_response(text)
    # rules
    if response:
        stats['generative'] += 1
        return response
    
    # Generate answer
    
    # stub - если нет ни одного ответа!
    stats['stub'] += 1
    return get_failure_phrases() 

In [20]:
# https://github.com/python-telegram-bot/python-telegram-bot
! pip install python-telegram-bot

In [21]:
# https://github.com/python-telegram-bot/python-telegram-bot/blob/master/examples/echobot.py

from telegram.ext import Updater, CommandHandler, MessageHandler, Filters


def start(update, context):
    """Send a message when the command /start is issued."""
    update.message.reply_text('Hi!')


def help_command(update, context):
    """Send a message when the command /help is issued."""
    update.message.reply_text('Help!')


def bot_answer(update, context):
    """Echo the user message."""
    question = update.message.text
    answer = bot(question)
    print(question, answer)
    print(stats)
    print()
    update.message.reply_text(answer)


def main():
    """Start the bot."""
    # добавляем свой token
    updater = Updater("***********************************************", use_context=True)

    dp = updater.dispatcher
    dp.add_handler(CommandHandler("start", start))
    dp.add_handler(CommandHandler("help", help_command))
    dp.add_handler(MessageHandler(Filters.text & ~Filters.command, bot_answer))

    # Start the Bot
    updater.start_polling()
    updater.idle()

In [22]:
main()

Очистить Не поняла вас
{'intent': 0, 'generative': 0, 'stub': 1}

Привет здарова
{'intent': 1, 'generative': 0, 'stub': 1}

Как жизнь? Прекрасный выбор для Вас!
{'intent': 2, 'generative': 0, 'stub': 1}

Что? Штож, я пытался это понять
{'intent': 2, 'generative': 0, 'stub': 2}

Эммм Не поняла Вас
{'intent': 2, 'generative': 0, 'stub': 3}

Отстань Идемте!..
{'intent': 2, 'generative': 1, 'stub': 3}

ывв ПОпробуйте еще раз!
{'intent': 2, 'generative': 1, 'stub': 4}

ыв Пожалуйста, повторите фразу
{'intent': 2, 'generative': 1, 'stub': 5}

